# BC rules phase 1

In [103]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

In [104]:
import os

BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/BR/Surgeries/data/'
STORE_ID = 1327
ACTUAL_RULE = 5
print([x for x in os.listdir(BASE_DIR) if x.startswith(str(STORE_ID))])

['1327_FR.csv', '1327_actual_00.csv', '1327_integ_210424.csv', '1327_catalog_01.csv', '1327_catalog_03.csv', '1327_catalog_02.csv', '1327_supply_03.csv', '1327_supply_02.csv', '1327_input.csv', '1327_supply_01.csv', '1327_OOS.csv', '1327_orders_00.csv']


## Import

In [105]:
import re
import pandas as pd
from datetime import datetime

dfs = {}
for df in ['actual','orders','catalog','supply','integ']:
    dfs[df] = pd.DataFrame()
    for i in [re.findall(r'_(\d+)',x)[-1] for x in os.listdir(BASE_DIR)
              if x.startswith(f'{STORE_ID}_{df}')]:
        aux = pd.read_csv(os.path.join(BASE_DIR,f'{STORE_ID}_{df}_{i}.csv'),
                          index_col= (0 if df=='orders' else None),
                          sep=(';' if df=='integ' else ',')
                         )
        aux['n_file'] = i
        dfs[df] = dfs[df].append(aux,ignore_index=True)
    dfs[df].columns = [x.lower().strip().replace(' ','_') for x in dfs[df].columns]
    print(df.title())
    display(dfs[df].sample())

Actual


,internal_branch_id,supply_category,stock,n_file
19,179,pas / leite,25,00


Orders


,store_id,shopper_branch_id,product_id,productbranch_id,name,package,order_id,units_requested,units_found,day_date,n_file
170,1327,7069,2685891,72927675,Salame italiano pequeno,Embalagem de 300g aprox.,32515076,0.3,0.294,2021-04-24,00


Catalog


,product_id,category_id,catalog_category_name,branch_id,productbranch_id,rank,n_file
38965,2706180,1200.0,Flip flops,26642,123888940,1,01


Supply


,store_id,catalog_store_branch_id,internal_branch_id,catalog_product_id,supply_category,sku,max_last_visited,rank,n_file
242446,1327,16456,394,1198743,perfumaria / papel higienico,5638216,2021-05-04T16:51:59.175187Z,3,03


Integ


,sku,branch_id,stock,price,n_file
923649,6112544,143,0,4.99,210424


res = pd.read_csv(os.path.join(BASE_DIR,'resultado.csv'))
res.sample()

## Functions

In [106]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [107]:
def gen_series(data,serie,col,cols,val_cols):
    df = data.copy()
    serie = sorted(set([df[col].min()-.1,df[col].max()+.1] + [x for x in serie]))
    df[f'{col}_threshold'] = [int(x.left) for x in pd.cut(df[col],bins=serie,right=False)]
    df[col]=1
    val_cols += [col]
    tot = df.groupby(cols)[val_cols].sum()
    df = df.groupby(cols+[f'{col}_threshold'])[val_cols].sum()
    df = df.groupby(level=list(range(len(cols)))).cumsum().reset_index().set_index(cols)
    df = df.join(tot.rename(columns={x:f'{x}_total' for x in val_cols}))
    df = df.join(data[data[col]==0].groupby(cols)[col].count().to_frame()\
                 .rename(columns={col:'products_with_stock_0'}))
    for c in val_cols:
        df[f'{c}_above_threshold'] = df[f'{c}_total'] - df[c]
        df[c] /= df[f'{c}_total']
    df = df.rename(columns={x:f'{x}_under_threshold' for x in val_cols})
    for th in ['under_threshold','total','above_threshold']:
        df[f'fr_{th}'] = df[f'units_found_{th}'] / (df[f'units_requested_{th}']+1e-8)
        df.drop(columns=[f'units_found_{th}',f'units_requested_{th}'],inplace=True)
    return df.fillna(0).reset_index()

## TAD

### OOS

In [108]:
oos = dfs['catalog'].merge(dfs['supply'],
                          left_on=['branch_id','product_id'],
                          right_on=['catalog_store_branch_id','catalog_product_id']
                         ).merge(dfs['integ'],
                                 left_on=['internal_branch_id','sku'],
                                 right_on=['branch_id','sku'])
oos.columns

Index(['product_id', 'category_id', 'catalog_category_name', 'branch_id_x',
       'productbranch_id', 'rank_x', 'n_file_x', 'store_id',
       'catalog_store_branch_id', 'internal_branch_id', 'catalog_product_id',
       'supply_category', 'sku', 'max_last_visited', 'rank_y', 'n_file_y',
       'branch_id_y', 'stock', 'price', 'n_file'],
      dtype='object')

In [109]:
oos.rename(columns={'catalog_category_name':'category_name',
                   'branch_id_x':'branch_id',
                   'n_file':'date'},inplace=True)
oos = oos[['store_id','branch_id','sku','product_id','category_id','category_name',
         'supply_category','max_last_visited','date','stock','price']].copy()
oos['max_last_visited'] = pd.to_datetime(oos['max_last_visited']).dt.date
oos['date'] = pd.to_datetime(oos['date'],format='%y%m%d').astype(str)
oos['category_id'] = oos['category_id'].fillna(0).astype(int)
oos.sample()

,store_id,branch_id,sku,product_id,category_id,category_name,supply_category,max_last_visited,date,stock,price
303714,1327,16456,5673461,1581543,141,Appetizers & Sides,pas / congelado salgado,2021-05-04,2021-04-24,13,26.99


### Catalog

In [110]:
cat = oos.groupby(['product_id','category_id','category_name','supply_category']
                 )['sku'].count().reset_index()
cat.sample()

,product_id,category_id,category_name,supply_category,sku
22498,2401215,1583,Toilet paper,perfumaria / papel higienico,3


### Orders

In [111]:
fr = dfs['orders'].merge(cat,how='left',on='product_id').fillna(0).astype(int,errors='ignore')
fr.rename(columns={'day_date':'date','shopper_branch_id':'branch_id'},inplace=True)
fr.insert(0,'orders',1)
fr.sample()

,orders,store_id,branch_id,product_id,productbranch_id,name,package,order_id,units_requested,units_found,date,n_file,category_id,category_name,supply_category,sku
491,1,1327,7483,2522559,68766725,Paprica defumada,50g,32453219,1,1,2021-04-24,0,75,Spices & Seasoning,seca / especiarias,65


In [132]:
fr['date'].value_counts().to_frame().sort_index()

,date
2021-04-21,81
2021-04-22,58
2021-04-23,93
2021-04-24,103
2021-04-25,69
2021-04-26,96


### Series

In [124]:
cols = ['date','store_id','branch_id',
        #'category_id','category_name',
        'supply_category']
df = oos.merge(fr[cols+['product_id','orders','units_requested','units_found']],
               how='left',
               on=cols+['product_id']).fillna(0)
df.sample()

,store_id,branch_id,sku,product_id,category_id,category_name,supply_category,max_last_visited,date,stock,price,orders,units_requested,units_found
447037,1327,6574,5760330,1575647,29,Butter,pas / manteiga,2021-05-04,2021-04-24,38,7.39,0.0,0.0,0.0


In [125]:
df = gen_series(df,range(10,61,10),'stock',cols,['orders','units_requested','units_found'])
df.head(6)

,date,store_id,branch_id,supply_category,stock_threshold,orders_under_threshold,stock_under_threshold,orders_total,stock_total,products_with_stock_0,orders_above_threshold,stock_above_threshold,fr_under_threshold,fr_total,fr_above_threshold
0,2021-04-24,1327,4026,acougue / aves,0,0.0,0.238462,0.0,130,11.0,0.0,99,0.0,0.0,0.0
1,2021-04-24,1327,4026,acougue / aves,10,0.0,0.376923,0.0,130,11.0,0.0,81,0.0,0.0,0.0
2,2021-04-24,1327,4026,acougue / aves,20,0.0,0.561538,0.0,130,11.0,0.0,57,0.0,0.0,0.0
3,2021-04-24,1327,4026,acougue / aves,30,0.0,0.676923,0.0,130,11.0,0.0,42,0.0,0.0,0.0
4,2021-04-24,1327,4026,acougue / aves,40,0.0,0.730769,0.0,130,11.0,0.0,35,0.0,0.0,0.0
5,2021-04-24,1327,4026,acougue / aves,50,0.0,0.815385,0.0,130,11.0,0.0,24,0.0,0.0,0.0


In [126]:
test = oos[(oos['branch_id']==4026)
           &(oos['supply_category']=='acougue / aves')]

for i in range(10,61,10):
    print(len(test[test['stock']<i])/len(test))

0.23846153846153847
0.3769230769230769
0.5615384615384615
0.676923076923077
0.7307692307692307
0.8153846153846154


## Export

In [12]:
df.to_csv(os.path.join(BASE_DIR,f'{STORE_ID}_input.csv'),index=False)

## End

In [13]:
time_exp(time.time() - start)
tono()

0 minutos con 15.34 segundos


In [134]:
np.percentile([30,40,20,20,30,20],80)

30.0